# Final Project

# Using linear programming for diet planing

## Meshkat Ahmadi
# Student ID 401422004

Here I wrote the utility functions to get calculated needed nutritions based on the weight
and I calculate the ideal weight based on BMI

In [6]:
import pandas as pd
from pulp import *
import re

patt=re.compile(r'[^A-Za-z0-9]+')
def convert_dict(food_dict):
    converted_dict = {}
    for nutrient in food_dict:
        for food in food_dict[nutrient]:
            new_food_name = re.sub(patt, '_', food)
            if new_food_name not in converted_dict:
                converted_dict[new_food_name] = {}
            if nutrient in ['Calories', 'Protein', 'TotalFat', 'Carbohydrate']:
                converted_dict[new_food_name][nutrient] = food_dict[nutrient][food]
    return converted_dict


def calculate_ideal_weight(height, gender):
    # Calculate ideal weight based on height and gender
    if gender.lower() == 'male':
        ideal_weight = 50 + 2.3 * ((height * 100) - 152.4) / 2.54
    elif gender.lower() == 'female':
        ideal_weight = 45.5 + 2.3 * ((height * 100) - 152.4) / 2.54
    else:
        raise ValueError("Invalid gender. Please enter 'male' or 'female'.")

    return ideal_weight

def calculate_nutrient_ranges(weight):
    
    # Calculate recommended nutrient ranges based on weight
    calories_range = (weight * 25, weight * 30)  # kilocalories
    protein_range = (weight * 0.8, weight * 1.2)  # grams
    fat_range = (weight * 0.5, weight * 0.7)  # grams
    carbohydrate_range = (weight * 3, weight * 4)  # grams
    sodium_range = (weight * 1.5, weight * 2.3)  # grams
    saturated_fat_range = (weight * 0.08, weight * 0.1)  # grams
    cholesterol_range = (weight * 0.15, weight * 0.2)  # grams
    sugar_range = (weight * 0.5, weight * 1)  # grams
    calcium_range = (weight * 0.015, weight * 0.03)  # grams
    iron_range = (weight * 0.005, weight * 0.01)  # grams
    potassium_range = (weight * 0.3, weight * 0.4)  # grams
    vitamin_c_range = (weight * 30, weight * 40)  # milligrams
    vitamin_e_range = (weight * 5, weight * 10)  # milligrams
    vitamin_d_range = (weight * 5, weight * 10)  # micrograms

    nutrient_ranges = {
        'Calories': (calories_range, 'kilocalories'),
        'Protein': (protein_range, 'grams'),
        'TotalFat': (fat_range, 'grams'),
        'Carbohydrate': (carbohydrate_range, 'grams'),
    }    

    return nutrient_ranges

# Create a food dictionary of Values

In [7]:
import pandas as pd
# parsing the food item
df = pd.read_csv('USDA.csv')
# removing the NAN records
df=df.dropna(subset=['Calories','Protein','Carbohydrate','TotalFat'], how='all')
# Create a list of the food items
food_items = list(df['Description'])
food_dict = dict()
for prop in df.columns[2::].tolist():
    food_dict[prop]=dict(zip(food_items,df[prop]))
food_nutrient = convert_dict(food_dict)

In [8]:
food_nutrient

{'BUTTER_WITH_SALT': {'Calories': 717.0,
  'Protein': 0.85,
  'TotalFat': 81.11,
  'Carbohydrate': 0.06},
 'BUTTER_WHIPPED_WITH_SALT': {'Calories': 717.0,
  'Protein': 0.85,
  'TotalFat': 81.11,
  'Carbohydrate': 0.06},
 'BUTTER_OIL_ANHYDROUS': {'Calories': 876.0,
  'Protein': 0.28,
  'TotalFat': 99.48,
  'Carbohydrate': 0.0},
 'CHEESE_BLUE': {'Calories': 353.0,
  'Protein': 21.4,
  'TotalFat': 28.74,
  'Carbohydrate': 2.34},
 'CHEESE_BRICK': {'Calories': 371.0,
  'Protein': 23.24,
  'TotalFat': 29.68,
  'Carbohydrate': 2.79},
 'CHEESE_BRIE': {'Calories': 334.0,
  'Protein': 20.75,
  'TotalFat': 27.68,
  'Carbohydrate': 0.45},
 'CHEESE_CAMEMBERT': {'Calories': 300.0,
  'Protein': 19.8,
  'TotalFat': 24.26,
  'Carbohydrate': 0.46},
 'CHEESE_CARAWAY': {'Calories': 376.0,
  'Protein': 25.18,
  'TotalFat': 29.2,
  'Carbohydrate': 3.06},
 'CHEESE_CHEDDAR': {'Calories': 403.0,
  'Protein': 24.9,
  'TotalFat': 33.14,
  'Carbohydrate': 1.28},
 'CHEESE_CHESHIRE': {'Calories': 387.0,
  'Protein'

# Calculation of the optimal weight and the recommended ranges

In [10]:
# Get user input
height = float(input("Enter height in meters: "))
weight = float(input("Enter current weight in kg: "))
age = int(input("Enter age in years: "))
gender = input("Enter gender (male/female): ")

# Calculate ideal weight and nutrient ranges
ideal_weight = calculate_ideal_weight(height, gender)
nutrient_ranges = calculate_nutrient_ranges(weight)

# Print the results
print("Ideal Weight: {:.2f} kg".format(ideal_weight))
print('----------------------------------------------------------')
print("Recommended Nutrient Ranges per day:")
for nutrient, (nutrient_range, unit) in nutrient_ranges.items():
    print("{}: {:.2f} - {:.2f} {}".format(nutrient, nutrient_range[0], nutrient_range[1], unit))
    
final_ranges = {}
for x,y in nutrient_ranges.items():
    final_ranges[x]=y[0]
    
nutrient_ranges = final_ranges

Enter height in meters: 1.7
Enter current weight in kg: 89
Enter age in years: 33
Enter gender (male/female): male
Ideal Weight: 65.94 kg
----------------------------------------------------------
Recommended Nutrient Ranges per day:
Calories: 2225.00 - 2670.00 kilocalories
Protein: 71.20 - 106.80 grams
TotalFat: 44.50 - 62.30 grams
Carbohydrate: 267.00 - 356.00 grams


# Let's plan for it

In [11]:
# the amount calclated for a day
nutrient_ranges

{'Calories': (2225.0, 2670.0),
 'Protein': (71.2, 106.8),
 'TotalFat': (44.5, 62.3),
 'Carbohydrate': (267.0, 356.0)}

In [12]:
# dict of food values
food_nutrient

{'BUTTER_WITH_SALT': {'Calories': 717.0,
  'Protein': 0.85,
  'TotalFat': 81.11,
  'Carbohydrate': 0.06},
 'BUTTER_WHIPPED_WITH_SALT': {'Calories': 717.0,
  'Protein': 0.85,
  'TotalFat': 81.11,
  'Carbohydrate': 0.06},
 'BUTTER_OIL_ANHYDROUS': {'Calories': 876.0,
  'Protein': 0.28,
  'TotalFat': 99.48,
  'Carbohydrate': 0.0},
 'CHEESE_BLUE': {'Calories': 353.0,
  'Protein': 21.4,
  'TotalFat': 28.74,
  'Carbohydrate': 2.34},
 'CHEESE_BRICK': {'Calories': 371.0,
  'Protein': 23.24,
  'TotalFat': 29.68,
  'Carbohydrate': 2.79},
 'CHEESE_BRIE': {'Calories': 334.0,
  'Protein': 20.75,
  'TotalFat': 27.68,
  'Carbohydrate': 0.45},
 'CHEESE_CAMEMBERT': {'Calories': 300.0,
  'Protein': 19.8,
  'TotalFat': 24.26,
  'Carbohydrate': 0.46},
 'CHEESE_CARAWAY': {'Calories': 376.0,
  'Protein': 25.18,
  'TotalFat': 29.2,
  'Carbohydrate': 3.06},
 'CHEESE_CHEDDAR': {'Calories': 403.0,
  'Protein': 24.9,
  'TotalFat': 33.14,
  'Carbohydrate': 1.28},
 'CHEESE_CHESHIRE': {'Calories': 387.0,
  'Protein'

In [61]:
from pulp import *

def generate_meal_plan(food_nutrients, nutrient_ranges, num_meals = 3, num_snacks = 2):
    # Create a list of food items
    food_items = list(food_nutrients.keys())
    # Create the LP problem
    problem = LpProblem("MealPlan", LpMinimize)

    # Create a dictionary of food variables
    food_vars = LpVariable.dicts("Food", food_items, lowBound=0, cat='Integer')

    # Add nutrient constraints
    for nutrient in nutrient_ranges:
        problem += lpSum([food_nutrients[food][nutrient] * food_vars[food] for food in food_items]) >= nutrient_ranges[nutrient][0], f"Min_{nutrient}"
        problem += lpSum([food_nutrients[food][nutrient] * food_vars[food] for food in food_items]) <= nutrient_ranges[nutrient][1], f"Max_{nutrient}"

    # Add the meal constraints
    problem += lpSum([food_vars[food] for food in food_items]) == num_meals, "Num_Meals"

    # Add the snack constraints
    problem += lpSum([food_vars[food] for food in food_items]) == num_snacks, "Num_Snacks"

    # Solve the problem
    problem.solve()

    # Print the status of the solution
    print("Status:", LpStatus[problem.status])

    # Print the recommended meal plan
    print("Recommended Meal Plan:")
    for food in food_items:
        if food_vars[food].value() > 0:
            print(food, ":", food_vars[food].value())


In [ ]:
# It seems that solver has a problem in linux I checked my implementation many times
generate_meal_plan(food_nutrient, nutrient_ranges)

In [ ]:
# Unfortunately solver has a problem with linux I checked the code extensively